# Задание
Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
Cохранить ее в pandas dataframe
Cохранить pandas dataframe в excel
Cохранить pandas dataframe в pickle
Cохранить pandas dataframe в БД

In [9]:
import requests
import json
import os
import pandas as pd
import sqlite3
import time
from tqdm.notebook import tqdm

# Список объектов

In [12]:
# Для скачивания данных по

offset_ = 1
limit_ = 1000
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
res = requests.get(url)
objects_data = res.json()
objects_data.get('data').get('list')[0]

{'objId': 13383,
 'developer': {'devId': 306,
  'shortName': '2МЕН ГРУПП',
  'fullName': '2МЕН ГРУПП',
  'groupName': '2МЕН ГРУПП ДЕВЕЛОПМЕНТ',
  'orgForm': {'id': 1, 'fullForm': 'Акционерное общество', 'shortForm': 'АО'},
  'companyGroup': 5776,
  'devInn': '7701651356'},
 'rpdRegionCd': 72,
 'objAddr': 'г Тюмень',
 'objFloorMin': 4,
 'objFloorMax': 20,
 'objElemLivingCnt': 68,
 'objReady100PercDt': '2022-12-31',
 'objSquareLiving': 6293.78,
 'objCompensFundFlg': 1,
 'objProblemFlg': 0,
 'latitude': 57.1519,
 'longitude': 65.5536,
 'problemFlag': 'NONE',
 'siteStatus': 'Строящиеся',
 'objGuarantyEscrowFlg': 0,
 'residentialBuildings': 1,
 'rpdNum': '72-000095',
 'pdId': 28105,
 'buildType': 'Жилое',
 'objStatus': 0}

In [23]:
limit_ = 100
objlist = []
import time
for i in tqdm(range(1, 11000, limit_)):
    try:
        offset_ = i
        url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
        res = requests.get(url)
        objlist.append(res.json())
    except JSONDecodeError as e:
        print(e)
        time.sleep(4)
        url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
        res = requests.get(url)
        objlist.append(res.json())

  0%|          | 0/110 [00:00<?, ?it/s]

In [24]:
objects_list = objects_data.get('data').get('list')
objids = [x.get('objId') for x in objects_list]
objids[:10]

[13383, 13397, 13391, 13398, 13399, 36912, 44076, 27476, 10891, 39176]

In [25]:
len(objlist)

110

In [26]:
object_ids = []
for el in objlist:
    object_ids.extend(el.get('data').get('list'))

In [27]:
len(object_ids)

10619

In [28]:
objects_list = objects_data.get('data').get('list')
objids = [x.get('objId') for x in objects_list]
objids[:10]

[13383, 13397, 13391, 13398, 13399, 36912, 44076, 27476, 10891, 39176]

# Данные по определенному объекту

In [29]:
# Для скачивания данных по определенному объекту, нужно добавить его id в конец строки запроса
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/29679'

In [30]:
res = requests.get(url)
res.json()

{'data': {'id': 29679,
  'pdId': 124647,
  'developer': {'devId': 7424,
   'devShortCleanNm': '212-2',
   'devShortNm': '212-2',
   'devFullCleanNm': '212-2',
   'problObjCnt': 0,
   'buildObjCnt': 1,
   'comissObjCnt': 0,
   'regRegionDesc': 'Ростовская область',
   'devPhoneNum': '+7(961)285-90-10',
   'devSite': 'taganrogjsk.wixsite.com',
   'devEmail': 'kirpol76@gmail.com',
   'devInn': '6154572922',
   'devOgrn': '1116154005923',
   'devKpp': '615401001',
   'devLegalAddr': 'Ростовская область, город Таганрог, улица Свободы/Трудовых резервов дом 25/9',
   'devFactAddr': 'обл Ростовская город Таганрог улица Сергея Шило д. 212-2',
   'lastRpdId': 126093,
   'fundGuarantyFlg': 0,
   'devOrgRegRegionCd': 61,
   'devEmplMainFullNm': 'Снимщикова Ольга Викторовна',
   'orgForm': {'id': 20,
    'fullForm': 'Жилищный или жилищно-строительный кооператив',
    'shortForm': 'ЖСК'},
   'objGuarantyEscrowFlg': 0,
   'govFundFlg': 0},
  'region': 61,
  'address': 'г Таганрог',
  'objElemLivingCn

In [31]:
import pandas as pd
from tqdm.auto import tqdm

In [32]:
d = []
for obj in tqdm(objids):
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{obj}'
    res = requests.get(url)
    d.append(res.json())

  0%|          | 0/1000 [00:00<?, ?it/s]

In [33]:
len(d)

1000

In [34]:
df=pd.json_normalize(d)

In [35]:
df.fillna(0, inplace=True)

In [36]:
df

,errcode,data.id,data.pdId,data.developer.devId,data.developer.devShortCleanNm,data.developer.devShortNm,data.developer.devFullCleanNm,data.developer.problObjCnt,data.developer.buildObjCnt,data.developer.comissObjCnt,...,data.developer.bankruptStage.bankruptStageDesc,data.developer.bankruptStage.bankruptLawUrl,data.developer.orgBankruptMsgDttm,data.metro.id,data.metro.name,data.metro.line,data.metro.color,data.metro.time,data.metro.isWalk,data.metro.colors
0,0,13383,28105,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,...,0,0,0,0,0,0,0,0.0,0,0
1,0,13397,28141,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,...,0,0,0,0,0,0,0,0.0,0,0
2,0,13391,28101,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,...,0,0,0,0,0,0,0,0.0,0,0
3,0,13398,28141,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,...,0,0,0,0,0,0,0,0.0,0,0
4,0,13399,28141,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,...,0,0,0,0,0,0,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,41271,473855,12144,КАПСТРОЙ,КАПСТРОЙ,КАПСТРОЙ,0,3,0,...,0,0,0,0,0,0,0,0.0,0,0
996,0,26466,89998,6970,КАРА+,КАРА+,КАРА+,0,1,0,...,0,0,0,0,0,0,0,0.0,0,0
997,0,15822,31564,3332,КАРГО-ТРЕЙД,КАРГО-ТРЕЙД,КАРГО-ТРЕЙД,0,1,1,...,0,0,0,0,0,0,0,0.0,0,0
998,0,38862,406238,11254,КАСКАД СЗ,КАСКАД СЗ,КАСКАД СПЕЦИАЛИЗИРОВАННЫЙ ЗАСТРОЙЩИК,0,3,0,...,0,0,0,0,0,0,0,0.0,0,0


# Запись файлов :

In [38]:
def save_df_as_sqlite(df, fname="objects_data"):
    print(f"Запись Датафрейма в файл {fname}.sqlite")
    df = df.astype(str)
    if "data" not in os.listdir():
        os.mkdir(f"./data")
    conn = sqlite3.connect(f"./data/{fname}.sqlite")
    df.to_sql(fname, conn, if_exists="replace", index=False)
    print(f"Файл {fname}.sqlite успешно записан\n---\n")
    
    
def save_df_as_xlsx(df, fname="objects_data"):
    print(f"Запись Датафрейма в файл {fname}.xlsx")
    df.to_excel(f"./data/{fname}.xlsx", engine="openpyxl")
    print(f"Файл {fname}.sqlite успешно записан\n---\n")

    
def save_df_as_pickle(df, fname="objects_data"):
    print(f"Запись Датафрейма в файл {fname}.pkl")
    df.to_pickle(f"./data/{fname}.pkl")
    print(f"Файл {fname}.sqlite успешно записан")

In [39]:
# Запись df в файлы
save_df_as_sqlite(df)
save_df_as_xlsx(df)
save_df_as_pickle(df)

Запись Датафрейма в файл objects_data.sqlite
Файл objects_data.sqlite успешно записан
---

Запись Датафрейма в файл objects_data.xlsx
Файл objects_data.sqlite успешно записан
---

Запись Датафрейма в файл objects_data.pkl
Файл objects_data.sqlite успешно записан
